In [ ]:
!pip install transformers==4.31.0
!pip install pytorch-crf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.0
    Uninstalling transformers-4.34.0:
      Successfully uninstalled transformers-4.34.0


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:
print(transformers.__version__)

4.31.0


In [ ]:
import joblib
import torch
import torch.nn as nn
import transformers
from torchcrf import CRF
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn import model_selection

from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, logging,AdamW




In [ ]:
class config:
    MAX_LEN = 100
    TRAIN_BATCH_SIZE = 1
    VALID_BATCH_SIZE = 1
    BASE_MODEL_PATH = "vinai/phobert-base-v2"
    # MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/ner vietnam"
    TRAINING_FILE = "/content/drive/MyDrive/Colab Notebooks/vietnam ner/loc.csv"
    TESTING_FILE = "/content/drive/MyDrive/Colab Notebooks/vietnam ner/test_2016_fixed.csv"
    TOKENIZER = transformers.AutoTokenizer.from_pretrained(BASE_MODEL_PATH, do_lower_case=True)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
train_data = pd.read_csv(config.TRAINING_FILE)
train_data

,Word,Ner,Nested_ner,sentence
0,Rác,O,O,0
1,bị,O,O,0
2,vứt,O,O,0
3,khắp,O,O,0
4,khu_vực,O,O,0
...,...,...,...,...
1757,sau,O,O,100
1758,lễ_hội,O,O,100
1759,ở,O,O,100
1760,Anh,B-LOCATION-GPE,O,100


In [ ]:
train_data.isnull().any()

Word           True
Ner            True
Nested_ner     True
sentence      False
dtype: bool

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class EntityDataset:
    def __init__(self, texts, ner, nest_ner):
        self.texts = texts
        self.ner = ner
        self.nest_ner = nest_ner
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        ner = self.ner[item]
        nest_ner = self.nest_ner[item]

        ids = []
        ner = []
        nest_ner =[]
        mask=[]
        token_type_ids=[]


        inputs = config.TOKENIZER.encode(
                text,
                add_special_tokens=False
            )
        input_len = len(inputs)
        padding_len =config.MAX_LEN - input_len
        ids.extend(inputs + [0]*padding_len )
        ner.extend(ner+ [37]*padding_len)
        nest_ner.extend(nest_ner + [6]*padding_len)

        mask.extend([1] * input_len + [0]*padding_len)
        token_type_ids.extend([0]*config.MAX_LEN)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "ner": torch.tensor(ner, dtype=torch.long),
            "nest_ner": torch.tensor(nest_ner, dtype=torch.long),
        }

In [ ]:
# def process_data(data_path):
#     df = pd.read_csv(data_path,encoding='utf-16')

#     df.dropna(inplace=True)

#     enc_tag = preprocessing.LabelEncoder()
#     enc_pos = preprocessing.LabelEncoder()
#     df.loc[:, "TAG"] = enc_tag.fit_transform(df["TAG"])
#     df.loc[:, "POS1"] = enc_pos.fit_transform(df["POS1"])

#     sentences = df.groupby("Sentences")["Word"].apply(list).values
#     tag = df.groupby("Sentences")["TAG"].apply(list).values
#     pos= df.groupby("Sentences")["POS1"].apply(list).values
#     df.loc[:, "OriginalLabel"] = enc_tag.inverse_transform(df["TAG"])
#     df.loc[:, "OriginalPOS"] = enc_pos.inverse_transform(df["POS1"])


#     label_mapping = df[["TAG", "OriginalLabel"]].drop_duplicates().sort_values("TAG")
#     pos_mapping = df[["POS1", "OriginalPOS"]].drop_duplicates().sort_values("POS1")




#     return sentences, tag, enc_tag,pos,enc_pos

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_data['Ner'] = le.fit_transform(train_data['Ner'])

NameError: name 'train_data' is not defined

In [ ]:
def process_data(data_path):
    df = pd.read_csv(data_path)

    df.dropna(inplace=True)


    enc_ner = preprocessing.LabelEncoder()
    enc_nested_ner = preprocessing.LabelEncoder()
    df.loc[:, "Ner"] = enc_ner.fit_transform(df["Ner"])
    df.loc[:, "Nested_ner"] = enc_nested_ner.fit_transform(df["Nested_ner"])

    sentences = df.groupby("sentence")["Word"].apply(list).values
    ner = df.groupby("sentence")["Ner"].apply(list).values
    nested_ner= df.groupby("sentence")["Nested_ner"].apply(list).values
    df.loc[:, "OriginalNer"] = enc_ner.inverse_transform(df["Ner"].astype(int))
    df.loc[:, "OriginalNested"] = enc_nested_ner.inverse_transform(df["Nested_ner"].astype(int))


    # label_mapping = df[["TAG", "OriginalLabel"]].drop_duplicates().sort_values("Ner")
    # pos_mapping = df[["POS1", "OriginalPOS"]].drop_duplicates().sort_values("NESTED_NER")




    return sentences, ner, enc_ner,nested_ner,enc_nested_ner

In [ ]:
# def process_data(data_path):
#     df = pd.read_csv(data_path)

#     df.dropna(inplace=True)

#     enc_ner = preprocessing.LabelEncoder()
#     enc_nested_ner = preprocessing.LabelEncoder()

#     # Check if the 'Ner' column contains only integers
#     if not pd.api.types.is_numeric_dtype(df["Ner"]):
#         # Convert the 'Ner' column to integers
#         df["Ner"] = df["Ner"].astype(int)

#     df.loc[:, "Ner"] = enc_ner.fit_transform(df["Ner"])
#     df.loc[:, "Nested_ner"] = enc_nested_ner.fit_transform(df["Nested_ner"].astype(int))

#     sentences = df.groupby("sentence")["Word"].apply(list).values
#     ner = df.groupby("sentence")["Ner"].apply(list).values
#     nested_ner = df.groupby("sentence")["Nested_ner"].apply(list).values
#     df.loc[:, "OriginalNer"] = enc_ner.inverse_transform(df["Ner"])
#     df.loc[:, "OriginalNested"] = enc_nested_ner.inverse_transform(df["Nested_ner"])

#     return sentences, ner, enc_ner, nested_ner, enc_nested_ner

In [ ]:
# def process_data(data_path):
#     df = pd.read_csv(data_path)
#     df.dropna(inplace=True)
#     enc_tag = preprocessing.LabelEncoder()
#     enc_pos = preprocessing.LabelEncoder()
#     df.loc[:, "TAG"] = enc_tag.fit_transform(df["TAG"])
#     df.loc[:, "POS1"] = enc_pos.fit_transform(df["POS1"])
#     sentences = df.groupby("Sentences")["Word"].apply(list).values
#     tag = df.groupby("Sentences")["TAG"].apply(list).values
#     pos = df.groupby("Sentences")["POS1"].apply(list).values

#     # Create dictionaries for mapping
#     tag_mapping = dict(enumerate(enc_tag.classes_))
#     pos_mapping = dict(enumerate(enc_pos.classes_))

#     # Replace integer values with original string labels
#     df.loc[:, "OriginalLabel"] = df["TAG"].map(tag_mapping)
#     df.loc[:, "OriginalPOS"] = df["POS1"].map(pos_mapping)

#     return sentences, tag, enc_tag, pos, enc_pos

In [ ]:
train_sentences, ner, enc_ner,nested, enc_nested = process_data(config.TRAINING_FILE)

In [ ]:
train_dataset = EntityDataset(texts=train_sentences, ner=ner, nest_ner=nested)

In [ ]:
# test_sentences, test_tag, enc_tag_test,pos_tag_test, enc_pos_test = process_data(config.TESTING_FILE)

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences_train, train_sentences_test, ner_train, ner_test, nested_train, nested_test = train_test_split(train_sentences, ner, nested, test_size=0.2, random_state=42)

# No need to split encoders, just use their classes_
enc_ner_train_classes = enc_ner.classes_
enc_nested_train_classes = enc_nested.classes_



In [ ]:
train_dataset = EntityDataset(texts=train_sentences_train, ner=ner_train, nest_ner=nested_train)

In [ ]:
test_dataset = EntityDataset(texts=train_sentences_test, ner=ner_test, nest_ner=nested_test)

In [ ]:
print(train_dataset[56])

{'ids': tensor([4473, 6834,   20, 2704,   27,    3,   19,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.TRAIN_BATCH_SIZE)

In [ ]:
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.VALID_BATCH_SIZE, num_workers=1)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
print("Mapping for NER:")
for label, encoded_value in zip(enc_ner.classes_, enc_ner.transform(enc_ner.classes_)):
    print(f"{label} -> {encoded_value}")

print("\nMapping for Nested_ner:")
for label, encoded_value in zip(enc_nested.classes_, enc_nested.transform(enc_nested.classes_)):
    print(f"{label} -> {encoded_value}")

Mapping for NER:
B-DATETIME -> 0
B-DATETIME-DATE -> 1
B-DATETIME-DURATION -> 2
B-DATETIME-TIMERANGE -> 3
B-EVENT -> 4
B-EVENT-CUL -> 5
B-LOCATION-GPE -> 6
B-LOCATION-STRUC -> 7
B-ORGANIZATION -> 8
B-PERSON -> 9
B-PERSONTYPE -> 10
B-PHONENUMBER -> 11
B-PRODUCT -> 12
B-PRODUCT-LEGAL -> 13
B-QUANTITY -> 14
B-QUANTITY-CUR -> 15
B-QUANTITY-DIM -> 16
B-QUANTITY-NUM -> 17
B-QUANTITY-ORD -> 18
B-URL -> 19
I-DATETIME -> 20
I-DATETIME-DATE -> 21
I-DATETIME-DURATION -> 22
I-EVENT -> 23
I-EVENT-CUL -> 24
I-LOCATION-GPE -> 25
I-LOCATION-STRUC -> 26
I-ORGANIZATION -> 27
I-PERSON -> 28
I-PERSONTYPE -> 29
I-PRODUCT -> 30
I-PRODUCT-LEGAL -> 31
I-QUANTITY -> 32
I-QUANTITY-CUR -> 33
I-QUANTITY-DIM -> 34
I-QUANTITY-ORD -> 35
I-URL -> 36
O -> 37

Mapping for Nested_ner:
B-DATETIME -> 0
B-DATETIME-DATE -> 1
B-LOCATION-GPE -> 2
B-QUANTITY-NUM -> 3
I-LOCATION-GPE -> 4
I-QUANTITY-NUM -> 5
O -> 6


In [ ]:
meta_data = {
    "enc_ner": enc_ner,
    "enc_nested_ner": enc_nested
}

joblib.dump(meta_data, "/content/drive/MyDrive/Colab Notebooks/vietnam ner/encode2.bin")

['/content/drive/MyDrive/Colab Notebooks/vietnam ner/encode2.bin']

In [ ]:
def train_fn(data_loader, model, optimizer,optimizer1, device, scheduler):
    model.train()
    pos_losses = []
    ner_losses = []

    predicted=[]
    real=[]
    predicted_pos=[]
    real_pos=[]



    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        optimizer.zero_grad()
        optimizer1.zero_grad()
        output_tag, output_pos, loss_tag,loss = model(**data)
        # pro.append(p)




        loss.backward(retain_graph=True)
        loss_tag.backward(retain_graph=True)

        for sublist_a, sublist_b in zip(output_tag, data["ner"]):
          n = len(sublist_a)
          first_n_values = sublist_b[:n]

          real.append(first_n_values)
          predicted.append(sublist_a)

        for sublist_a, sublist_b in zip(output_pos, data["nest_ner"]):
          n = len(sublist_a)
          first_n_values = sublist_b[:n]

          real_pos.append(first_n_values)
          predicted_pos.append(sublist_a)


        optimizer.step()
        optimizer1.step()
        scheduler.step()

        pos_losses.append(loss.item())
        ner_losses.append(loss_tag.item())

    avg_pos_loss = sum(pos_losses) / len(pos_losses)
    avg_ner_loss = sum(ner_losses) / len(ner_losses)

    y_true_tag = torch.tensor([label for sublist in real for label in sublist]).cpu()
    del real
    y_pred_tag = torch.tensor([label for sublist in predicted for label in sublist]).cpu()
    del predicted
    f1_tag = f1_score(y_true_tag, y_pred_tag, average='micro')

    y_true_pos = torch.tensor([label for sublist in real_pos for label in sublist]).cpu()
    del real_pos
    y_pred_pos = torch.tensor([label for sublist in predicted_pos for label in sublist]).cpu()
    del predicted_pos
    f1_pos = f1_score(y_true_pos, y_pred_pos, average='micro')


    return  avg_ner_loss,avg_pos_loss,f1_tag,f1_pos

def eval_fn(data_loader, model, device):
    model.eval()
    pos_losses = []
    ner_losses = []

    predicted=[]
    real=[]
    predicted_pos=[]
    real_pos=[]



    with torch.no_grad():
      for data in tqdm(data_loader, total=len(data_loader)):
          for k, v in data.items():
              data[k] = v.to(device)

          output_tag, output_pos, loss_tag,loss = model(**data)

          pos_losses.append(loss.item())
          ner_losses.append(loss_tag.item())
          for sublist_a, sublist_b in zip(output_tag, data["ner"]):
            n = len(sublist_a)
            first_n_values = sublist_b[:n]

            real.append(first_n_values)
            predicted.append(sublist_a)

          for sublist_a, sublist_b in zip(output_pos, data["nest_ner"]):
            n = len(sublist_a)
            first_n_values = sublist_b[:n]

            real_pos.append(first_n_values)
            predicted_pos.append(sublist_a)

      avg_pos_loss = sum(pos_losses) / len(pos_losses)
      avg_ner_loss = sum(ner_losses) / len(ner_losses)

      y_true_tag = torch.tensor([label for sublist in real for label in sublist]).cpu()
      del real
      y_pred_tag = torch.tensor([label for sublist in predicted for label in sublist]).cpu()
      del predicted
      f1_tag = f1_score(y_true_tag, y_pred_tag, average='micro')

      y_true_pos = torch.tensor([label for sublist in real_pos for label in sublist]).cpu()
      del real_pos
      y_pred_pos = torch.tensor([label for sublist in predicted_pos for label in sublist]).cpu()
      del predicted_pos
      f1_pos = f1_score(y_true_pos, y_pred_pos, average='micro')


    return avg_ner_loss,avg_pos_loss,f1_tag,f1_pos


In [ ]:
import torch.nn.functional as F
log_soft = F.log_softmax

In [ ]:
class EntityModel(nn.Module):
    def __init__(self, num_ner, num_nested):
        super(EntityModel, self).__init__()
        self.num_ner = num_ner
        self.num_nested = num_nested
        self.phobert = transformers.AutoModel.from_pretrained(config.BASE_MODEL_PATH,return_dict=False)
        # for param in self.phobert.parameters():
        #     param.requires_grad = False
        self.bert_drop_1 = nn.Dropout(0.3)
        self.bert_drop_2 = nn.Dropout(0.3)
        self.out_ner = nn.Linear(768, self.num_ner)
        self.out_nest = nn.Linear(768, self.num_nested)
        self.crf = CRF(num_ner, batch_first = True)
        self.crf_nest = CRF(num_nested, batch_first = True)
    def forward(
        self,
        ids,
        mask,
        token_type_ids,
        ner,
        nest_ner
    ):
        o1, _ = self.phobert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        bo_tag = self.bert_drop_1(o1)
        bo_nest = self.bert_drop_2(o1)

        tag = self.out_ner(bo_tag)
        nest = self.out_nest(bo_nest)



        if ner is not None:
            loss_tag = -self.crf(log_soft(tag, 2), ner, mask=mask.type(torch.uint8), reduction='mean')
            prediction = self.crf.decode(tag, mask=mask.type(torch.uint8))


        else:
            prediction = self.crf.decode(tag, mask=mask.type(torch.uint8))
            # return prediction

        if nest_ner is not None:
            loss_nested = -self.crf_nest(log_soft(nest, 2), nest_ner, nest_ner=mask.type(torch.uint8), reduction='mean')
            nested_prediction = self.crf_nest.decode(nest, mask=mask.type(torch.uint8))


        else:
            nested_prediction = self.crf_nest.decode(nest, mask=mask.type(torch.uint8))


        return prediction, nested_prediction,loss_tag,loss_nested

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
num_tag = len(list(enc_ner.classes_))
num_nest = len(list(enc_nested.classes_))

In [ ]:
model = EntityModel(num_ner=num_tag, num_nested=num_nest)
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EntityModel(
  (phobert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [ ]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
from torcheval.metrics.functional import multiclass_f1_score

In [ ]:
model = EntityModel(num_ner=num_tag, num_nested=num_nest)
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay
            )
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(
                nd in n for nd in no_decay
            )
        ],
        "weight_decay": 0,
    },
]

num_train_steps = int(
    len(train_sentences) / config.TRAIN_BATCH_SIZE * config.EPOCHS
)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps
)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: type object 'config' has no attribute 'EPOCHS'

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        optimizer.zero_grad()
        _, _, loss, acc, _ = model(**data)
        loss.backward()
        optimizer.step()
        scheduler.step()
        final_loss += loss.item()
    return final_loss / len(data_loader), acc
def loss_fn(output, target, mask, num_labels):
    lfn = nn.CrossEntropyLoss(ignore_index=1)
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    #torch.Size([batch_size*max_len]) torch.Size([batch_size*max_len, num_class])
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(1).type_as(target)
    )#batch_size*max_len
    loss = lfn(active_logits, active_labels)
    return loss
def accuracy_fn(output, target, mask, num_labels):
    _, predicted = torch.max(output, 2)
    #print(output.size(), predicted.size(), target.size())
    #torch.Size([16, 128, 22]) torch.Size([16, 128]) torch.Size([16, 128])
    correct = (predicted == target) & (target != 1) & (mask == 1)
    #print(predicted)
    #print(target)
    num_correct = correct.sum().item()
    num_samples = target[(target != 1) & (mask == 1)].size()[0]
    #print(num_correct, num_samples)
    accuracy = num_correct / num_samples if num_samples > 0 else 0.0
    return accuracy

def f1score_fn(output, target, mask, num_labels):
    #print(output.size(), predicted.size(), target.size())
    #torch.Size([batch_size, max_len, num_class]) torch.Size([batch_size, max_len]) torch.Size([batch_size, max_len])
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(1).type_as(target)
    )
    f1 = multiclass_f1_score(active_logits, active_labels, num_classes=num_labels, average="macro")
    return f1
train_losses = []
valid_losses = []
best_loss = np.inf
for epoch in range(config.EPOCHS):
    train_loss, train_acc = train_fn(
        train_data_loader,
        model,
        optimizer,
        device,
        scheduler
    )

    print(f"Epoch {epoch + 1}/{config.EPOCHS} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Epoch {epoch + 1}/{config.EPOCHS} - Valid Loss: {val_loss:.4f}, Valid Acc: {val_acc:.4f}, F1-score: {f1_score:.4f}")
    train_losses.append(train_loss)
    valid_losses.append(val_loss)
    if val_loss < best_loss:
        torch.save(model.state_dict(), config.MODEL_PATH)
        best_loss = val_loss

In [ ]:
for data in tqdm(train_data_loader, total=len(train_data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        output_tag, output_pos, loss_tag,loss = model(**data)
        break

  0%|          | 0/101 [00:02<?, ?it/s]


ValueError: the first two dimensions of emissions and tags must match, got (1, 100) and (1, 77)

In [ ]:
data

{'ids': tensor([[23132,    45,  4636,  1270,   160,   116,  1633,  1208,    35,   214,
          30284,     7,   157,    53,    26,   137,  3511,    18,    75,    97,
            895,   776,     5,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
 'mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0,

In [ ]:
output_tag, output_pos, loss_tag,loss = model(**data)

ValueError: the first two dimensions of emissions and tags must match, got (1, 100) and (1, 77)

In [ ]:
param_optimizer = list(model.named_parameters())
# no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
# no_decay = ['bias', 'gamma', 'beta','LayerNorm.weight','LayerNorm.bias']
no_decay = ['bias', 'gamma', 'beta',"LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay
            )
        ],
        "weight_decay": 0.001,
    }

]

num_train_steps = int(
    len(train_sentences) / config.TRAIN_BATCH_SIZE * 10
)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
optimizer1 = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps
)

best_loss = np.inf
# best_acc = 0.0
for epoch in range(10):
    train_ner_losses,train_pos_losses,f1_tag_train,f1_pos_train = train_fn(
        train_data_loader,
        model,
        optimizer,
        optimizer1,
        device,
        scheduler
    )

    # test_ner_losses,test_pos_losses,f1_tag_test,f1_pos_test = eval_fn(
    #     test_data_loader,
    #     model,
    #     device
    # )

    print(f"Train Nest Loss = {train_pos_losses:.4f}")
    print(f"Train NER Loss = {train_ner_losses:.4f}")
    # print(f"Valid Nest Loss = {test_pos_losses:.4f}")
    # print(f"Valid NER Loss = {test_ner_losses:.4f}")

    print(f"Train Nest F1 = {f1_pos_train:.4f}")
    print(f"Train NER f1 = {f1_tag_train:.4f}")
    # print(f"Valid Nest f1 = {f1_pos_test:.4f}")
    # print(f"Valid NER f1 = {f1_tag_test:.4f}")


    test=train_ner_losses + train_pos_losses
    if test < best_loss:
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/test_model/model.pth')
        print("Save model.")
        best_loss = test
# torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/test_model/best_model_crf3.pth')

  0%|          | 0/101 [00:00<?, ?it/s]


ValueError: the first two dimensions of emissions and tags must match, got (1, 100) and (1, 77)

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta', "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     "weight_decay": 0.001,
     }
]
num_train_steps = int(len(train_sentences) / config.TRAIN_BATCH_SIZE * 10)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
optimizer1 = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
best_loss = np.inf

for epoch in range(10):
    train_ner_losses, train_pos_losses, f1_tag_train, f1_pos_train = 0.0, 0.0, 0.0, 0.0
    for batch in train_data_loader:
        try:
            ner_loss, pos_loss, f1_tag, f1_pos = train_fn(batch, model, optimizer, optimizer1, device, scheduler)
            train_ner_losses += ner_loss
            train_pos_losses += pos_loss
            f1_tag_train += f1_tag
            f1_pos_train += f1_pos
        except Exception as e:
            print(f"Error encountered during training: {e}")
            print("Skipping batch...")
            continue

    train_ner_losses /= len(train_data_loader)
    train_pos_losses /= len(train_data_loader)
    f1_tag_train /= len(train_data_loader)
    f1_pos_train /= len(train_data_loader)

    print(f"Train Nest Loss = {train_pos_losses:.4f}")
    print(f"Train NER Loss = {train_ner_losses:.4f}")
    print(f"Train Nest F1 = {f1_pos_train:.4f}")
    print(f"Train NER f1 = {f1_tag_train:.4f}")

    test = train_ner_losses + train_pos_losses
    if test < best_loss:
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/test_model/model.pth')
        print("Save model.")
        best_loss = test

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/5 [00:00<?, ?it/s]


Error encountered during training: 'str' object has no attribute 'items'
Skipping batch...


  0%|          | 0/5 [00:00<?, ?it/s]


Error encountered during training: 'str' object has no attribute 'items'
Skipping batch...


  0%|          | 0/5 [00:00<?, ?it/s]


...

  0%|          | 0/5 [00:00<?, ?it/s]


Error encountered during training: 'str' object has no attribute 'items'
Skipping batch...


  0%|          | 0/5 [00:00<?, ?it/s]


Error encountered during training: 'str' object has no attribute 'items'
Skipping batch...
Train Nest Loss = 0.0000
Train NER Loss = 0.0000
Train Nest F1 = 0.0000
Train NER f1 = 0.0000


KeyboardInterrupt: 

In [ ]:
optimizer

AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 3e-05
    lr: 0.0
    weight_decay: 0.001
)

dfdsdfdffd